# System Dynamics
Parameters:

- $ r $: Radius of the wheels
- $ L $: Distance from the center of mass to each wheel (half the body width)

State Variables:

- $ x $: $ x $-position of the rover's center of mass
- $ y $: $ y $-position of the rover's center of mass
- $ \psi $: Absolute heading angle of the rover (orientation of the body)
- $ \phi_L $: Absolute angle of rotation of the left wheel
- $ \phi_R $: Absolute angle of rotation of the right wheel

Inputs:

- $ \dot{\phi}_L $: Angular velocity of the left wheel
- $ \dot{\phi}_R $: Angular velocity of the right wheel

The system of differential equations describing the rover is:
$$
\begin{align}
\dot{x} &= \frac{r}{2} (\dot{\phi}_R + \dot{\phi}_L) \cos(\psi) \\
\dot{y} &= \frac{r}{2} (\dot{\phi}_R + \dot{\phi}_L) \sin(\psi) \\
\dot{\psi} &= \frac{r}{L} (\dot{\phi}_R - \dot{\phi}_L) \\
\dot{\phi}_L &= u_L \\
\dot{\phi}_R &= u_R
\end{align}
$$

We can see that the current state of the system does not directly change the system (0 input = no movement), so $A = 0, \dot{z} = Bu$.  In matrix form, this is:
$$
z = \begin{bmatrix}
            x \\
            y \\
            \psi \\
            \phi_L \\
            \phi_R
        \end{bmatrix}, 
u = \begin{bmatrix}
        u_L \\
        u_R
    \end{bmatrix}
$$
$$ 
\dot{z} = B(\psi)u \\
A = 0, B = \begin{bmatrix}
            \frac{r}{2} \cos(\psi) & \frac{r}{2} \cos(\psi) \\
            \frac{r}{2} \sin(\psi) & \frac{r}{2} \sin(\psi) \\
            -\frac{r}{L} & \frac{r}{L} \\
            1 & 0 \\
            0 & 1
            \end{bmatrix}
$$
